In [8]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from housecredit import HouseCredit
from scraping import DemandDeposit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
HouseCredit('https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf', [6,7,8,9]).tokenize()

['Entrada em vigor: 21-Dez-2020    2.',
 'OPERAÇÕES DE CRÉDITO (PARTICULARES)                                                                                 ÍNDICE  2.1 Crédito à habitação e outros créditos hipotecários    Comissões  Acresce    Valor  Outras condições  Em %  Euros (Mín/Máx)  Imposto  anual  Produto: Crédito Habitação Regime Geral e Regime Deficientes.',
 'Caracterização do produto: Crédito à habitação com garantia hipotecária destinado à aquisição, construção e realização de obras em  habitação própria e permanente, secundária ou para arrendamento (no Regime Deficientes só habitação própria e permanente) ou à  aquisição ou manutenção de direitos de propriedade sobre terrenos ou edifícios já existentes ou projetados.',
 'Comissões iniciais  1.',
 'Reanálise  nan  250,00   nan  IVA – 23%  Nota (10)  0,00 no Regime  2.',
 'Comissão de formalização  nan  500,00  nan  IS – 4%  Deficientes  3.',
 'Avaliação        Valor devido  3.1  Imóvel residencial  nan  275,00  nan  IVA

In [10]:
DemandDeposit('https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf', [4]).values()

{'Emissão de extrato': ['Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2. Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3.'],
 'Fotocópias de segundas vias de talões de depósito': ['Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3. Manutenção de conta      3.1 Conta D.O.'],
 'Manutenção de conta': ['Manutenção de conta      3.1 Conta D.O. Particulares (Nota 2)      - Saldo médio até 1.000€  15,00 Trimestral  60,00  - Saldo médio entre 1.001€ e 2.500€  10,00 Trimestral  40,00  IS – 4%    - Saldo médio superior a 2.500€  0,00  nan  3.2 Conta Ordenado (Nota 1 e Nota 16)  0,00  nan  3.3 Conta Standard (não admite novas contratações)  0,00  nan  3.4 Conta Moeda Estrangeira (Nota 12)  5,00 Mensal  60,00  4.'],
 'Levantamento de numerário': ['Levantamento de numerário          7.1 Ao bal